## SPAM DETECTION using ML+NLP+DL

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
import joblib

# Step 2: Load your dataset

In [2]:
dataset = pd.read_csv("C:\\Users\\Vaibhav Singh\\Downloads\\spam.csv")  # yahan apni file ka naam likho

print(dataset.head())
print(dataset.info())

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


# Step 3: Label Encoding (ham → 0, spam → 1)

In [3]:
dataset['Category'] = dataset['Category'].map({'ham': 0, 'spam': 1})

print(dataset['Category'].value_counts())

Category
0    4825
1     747
Name: count, dtype: int64


# Step 4: Text Tokenization

In [4]:
vocab_size = 5000
maxlen = 100

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['Message'])

sequences = tokenizer.texts_to_sequences(dataset['Message'])
padded = pad_sequences(sequences, maxlen=maxlen, padding='post')

# save tokenizer
joblib.dump(tokenizer, 'spam_tokenizer.pkl')
print("Tokenizer saved")

Tokenizer saved


# Step 5: Train-Test Split

In [5]:
X = padded
y = dataset['Category'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(X_train.shape, X_test.shape)

(4457, 100) (1115, 100)


# Step 6: Build LSTM Model

In [6]:
model = Sequential([
    Embedding(vocab_size, 128, input_length=maxlen),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

c:\Users\Vaibhav Singh\anaconda3\envs\myayu\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Step 7: Train Model

In [7]:
history = model.fit(
    X_train, 
    y_train, 
    epochs=5, 
    batch_size=32, 
    validation_data=(X_test, y_test)
)

Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 72ms/step - accuracy: 0.9358 - loss: 0.1882 - val_accuracy: 0.9892 - val_loss: 0.0479
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - accuracy: 0.9919 - loss: 0.0379 - val_accuracy: 0.9901 - val_loss: 0.0468
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 80ms/step - accuracy: 0.9946 - loss: 0.0223 - val_accuracy: 0.9874 - val_loss: 0.0503
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 13s 89ms/step - accuracy: 0.9978 - loss: 0.0101 - val_accuracy: 0.9874 - val_loss: 0.0567
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - accuracy: 0.9989 - loss: 0.0038 - val_accuracy: 0.9865 - val_loss: 0.0930


# Step 8: Save Model

In [8]:
model.save("spam_lstm_model.h5")
print("Model saved successfully")

Model saved successfully


# PREDICTION CODE (Use your own messages)
# 🧪 Step 9: Load Model + Tokenizer

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib

model = load_model("spam_lstm_model.h5")
tokenizer = joblib.load("spam_tokenizer.pkl")

# Step 10: Prediction Function

In [10]:
def predict_spam(text):
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=100, padding='post')

    prob = model.predict(pad)[0][0]

    if prob >= 0.5:
        print("Spam Message ❌")
    else:
        print("Not Spam ✅")

    print("Probability:", prob)

# Step 11: Test with sample Real world text/messages

In [11]:
## SPAM messages

predict_spam("Congratulations! You have won a free iPhone 15. Click the link to claim now!")
predict_spam("Win Rs.10,000 cash prize. Send YES to 98765 now. Limited time offer!")
predict_spam("URGENT! Your bank account will be blocked. Verify your KYC now: www.fakebank.com")
predict_spam("You are selected for a lucky draw. Call now to receive your prize.")




## Real-World HAM / Not Spam messages

predict_spam("Hey bro, are we meeting today at 6 pm?")
predict_spam("Mom said dinner is ready. Come fast.")
predict_spam("Can you send me the project files?")
predict_spam("I'll call you later, I'm in a meeting.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step
Spam Message ❌
Probability: 0.9961873
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Spam Message ❌
Probability: 0.9965351
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Spam Message ❌
Probability: 0.99719226
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Spam Message ❌
Probability: 0.99916375
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Not Spam ✅
Probability: 6.7491837e-06
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Not Spam ✅
Probability: 1.4288446e-05
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Not Spam ✅
Probability: 4.483568e-05
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Not Spam ✅
Probability: 8.775621e-06


# hum apna dataset ka real row kaise test karinge

In [12]:
print(dataset['Message'].iloc[2]), print(dataset['Category'].iloc[2])

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
1


(None, None)

In [13]:
sample_text = dataset['Message'].iloc[2]
print("Message:", sample_text)

predict_spam(sample_text)

Message: Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Spam Message ❌
Probability: 0.9998045
